## Task 1:
Write test cases for this feature. If something is unclear, something that you would typically clarify with product or engineering before finishing your test cases please add those questions at the bottom of your test cases.


In [24]:
import pandas as pd
import numpy as np
import os

# Get Test Cases
df = pd.read_csv(r'C:\Users\raven\Desktop\Take_home\Test_Cases.csv') # Your Path. I attached the file as separate cvs
df 

,ID,Test Case,Steps,Expected Result,Status
0,1,Verify categories dropdown loads,Navigate to module → open categories dropdown,"Dropdown opens with listed options (ID, Person...",NaN
1,2,Verify multi-selection behavior,Select multiple categories (e.g. ID + Personal...,Both are selected (check marks shown) and thei...,NaN
2,3,Required fields appear for selected category,Select “ID”,ID field appears in table marked as required,NaN
3,4,Required and optional fields appear correctly,Select “Personal Info”,"First Name and Last Name marked required, Addr...",NaN
4,5,Shared fields only appear once,Select “Personal Info” + “General Info”,"First Name and Last Name appear only once, not...",NaN
5,6,Deselect category removes its fields,Select “Personal Info” → Deselect it,Fields from this category disappear unless als...,NaN
6,7,Contact Info: phone OR email validation,Select “Contact Info”,Must map at least one of phone or email to ena...,NaN
7,8,Category mapping validation,Select one category and leave required field u...,“Next Step” button remains disabled,NaN
8,9,Progression after required mappings,"Select a category, map all required fields",“Next Step” button becomes enabled,NaN
9,10,Dropdown search functionality,Use search input to filter categories,Only matching categories appear,NaN


## Questions to Clarify with Product or Engineering

1. Are there any max limits on how many categories a user can select at once?
2. For Contact Info, is it allowed to map both phone and email or just one?
3. What happens if the CRM source fields list is empty or doesn’t have a required field?
4. Are optional fields shown by default or only when user chooses to map them?
5. Should unmapped required fields show an error message or only disable “Next Step”?
6. Are dropdowns always populated locally or do some rely on API calls (so we can handle delays/spinners in tests)?

# Task 2: 
Based on your “behind the scenes knowledge” of the drop-downs in the product, write modular/reusable automated tests for dropdowns across the application within the following guidelines (use whatever syntax/pseudo-syntax you feel most comfortable with, but please identify it and if you know Cypress, please use that):  

In [25]:
# BDD file

with open(r'C:\Users\raven\Desktop\Take_home\script\BDD.feature') as file: # your path to file
    content = file.read()
    print(content)

Feature: Match user CRM data with external library using category dropdowns

Scenario: Select a single category from dropdown
Given the user is on the matching screen
When the user opens the category dropdown
And selects the "ID" category
Then the required field "SSN" should appear in the mapping table

Scenario: Select multiple categories from dropdown
Given the user is on the matching screen
When the user selects "Personal Info" and "General Info" categories
Then the required fields "First Name" and "Last Name" should appear only once
And the other required fields from both categories should be listed

Scenario: Deselect a category
Given "Personal Info" and "General Info" are selected
When the user deselects "General Info"
Then all fields from "General Info" are removed
But fields shared with "Personal Info" (First Name, Last Name) remain

Scenario: Display of required vs optional fields
Given the user selects the "Personal Info" category
When the field mapping table appears
Then it 

In [ ]:
from behave import given, when, then
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def wait_for_dropdown_open(context, dropdown_locator):
    WebDriverWait(context.browser, 10).until(
        EC.visibility_of_element_located(dropdown_locator),
        message="Dropdown did not open in time"
    )

def select_option(context, dropdown_locator, element_text):
    context.browser.find_element(*dropdown_locator).click()
    wait_for_dropdown_open(context, (By.XPATH, f"xpath_elemenet_1'{element_text}')]"))
    context.browser.find_element(By.XPATH, f"xpath_elemenet_1'{element_text}')]").click()

def map_field(context, field_name, crm_field):
    field_dropdown = (By.XPATH, f"xpath_element_2'{field_name}')]]//select")
    WebDriverWait(context.browser, 10).until(
        EC.element_to_be_clickable(field_dropdown),
    )
    context.browser.find_element(field_dropdown).click()
    crm_option = (By.XPATH, f"xpath_element_3'{crm_field}')]")
    WebDriverWait(context.browser, 10).until(
        EC.visibility_of_element_located(crm_option)
    )
    context.browser.find_element(*crm_option).click()

def get_visible_fields(context):
    rows = context.browser.find_elements(By.CSS_SELECTOR, "xpath_element_4")
    return [r.text.strip() for r in rows if r.text.strip()]

#Steps:

@given("the user is on the matching screen")
def step_user_on_screen(context):
    context.browser.get("URL")
    WebDriverWait(context.browser, 10).until(
        EC.presence_of_element_located((By.ID, "category-dropdown"))
    )

@when('the user opens the category dropdown')
def step_open_dropdown(context):
    context.browser.find_element(By.ID, "category-dropdown").click()
    wait_for_dropdown_open(context, (By.CSS_SELECTOR, ".dropdown-menu"))

@when('the user selects the "{category}" category')
def step_select_category(context, category):
    select_option(context, (By.ID, "category-dropdown"), category)

@then('the required field "{field}" should appear in the mapping table')
def step_check_field_present(context, field):
    fields = get_visible_fields(context)
    assert field in " ".join(fields), f"Expected {field} in table, found {fields}"

@when('the user deselects "{category}"')
def step_deselect_category(context, category):
    select_option(context, (By.ID, "category-dropdown"), category)

@then('all fields from "{category}" are removed')
def step_check_removed(context, category):
    category_fields = {
        "General Info": ["First Name", "Last Name", "Job", "Favorite Food", "Favorite Book"]
    }
    fields = get_visible_fields(context)
    for f in category_fields.get(category, []):
        assert f not in fields, f"{f} should have been removed but still present"

@then('the "Next" button should be enabled')
def step_next_enabled(context):
    btn = context.browser.find_element(By.ID, "next-btn")
    assert btn.is_enabled(), "Next button should be enabled"

@then('the "Next" button should be disabled')
def step_next_disabled(context):
    btn = context.browser.find_element(By.ID, "next-btn")
    assert not btn.is_enabled(), "Next button should be disabled"
